### **A1.3 Solución de problemas y selección de características**

En esta actividad se trabajará con la base de datos *"Calificaciones.csv"*. Se busca hacer una selección de variables y solución de problemas para poder crear un modelo de regresión lineal múltiple que pueda predecir las calificaciones.



---

A continuación se carga la base de datos llamada "A1.3 Calificaciones.csv" al espacio de trabajo usando la función importando la librería pandas y con la función read_csv. Posteriormente se utiliza la función print y head(10) para mostrar los primeros 10 datos de la base de datos y comprobar que funcionen correctamente.
Después de eso se usa la función `.dtypes` para conocer el tipo de variables que tiene esta base de datos. Por último, se usa la función `.shape`, para conocer las dimensiones de los datos y la función `.columns`, para imprimir los nombres de las variables.


In [201]:
import pandas as pd
datos = pd.read_csv('A1.3 Calificaciones.csv')
print(datos.head(10))
print(datos.dtypes)
print("Cantidad de filas:", datos.shape[0])
print(datos.columns)

  Escuela Sexo  Edad  HorasDeEstudio  Reprobadas Internet  Faltas  G1  G2  G3
0      GP    F    18               2           0       no       6   5   6   6
1      GP    F    17               2           0      yes       4   5   5   6
2      GP    F    15               2           3      yes      10   7   8  10
3      GP    F    15               3           0      yes       2  15  14  15
4      GP    F    16               2           0       no       4   6  10  10
5      GP    M    16               2           0      yes      10  15  15  15
6      GP    M    16               2           0      yes       0  12  12  11
7      GP    F    17               2           0       no       6   6   5   6
8      GP    M    15               2           0      yes       0  16  18  19
9      GP    M    15               2           0      yes       0  14  15  15
Escuela           object
Sexo              object
Edad               int64
HorasDeEstudio     int64
Reprobadas         int64
Internet         

Después de correr el código de bloque anterior se puede mencionar que la base de datos cuenta con 10 columnas con los siguientes nombres:
* Escuela (objeto): Nombre de la escuela; 'GP' o 'MS'
* Sexo (objeto): Femenino o Masculino; 'M' o 'F'
* Edad (entero): Edad del estudiante; de 15 a 22
* HorasDeEstudio (entero): Horas de estudio por semana; 1 - <2hrs = 1, 2 - 5hrs = 1, 5 - 10hrs = 3, >10hrs = 4.
* Reprobadas (entero): Número de materias reprobadas; 1-2 materias = n, >3 = 4
* Internet (objeto): Acceso a internet; 'yes' o 'no'
* Faltas (entero): Número de faltas a clases; de 0 a 93
* G1 (entero): Calificación de primer periodo; de 0 a 20
* G2 (entero): Calificación de segundo periodo; de 0 a 20
* G3 (entero): Calificación final; de 0 a 20.


La base de datos cuenta con 395 alumnos con información registrada.


Las variables cuantitativas en los datos son:
* Edad
* Reprobadas
* Faltas
* G1
* G2
* G3


Las variables cualitativas son:
* Escuela
* Sexo
* Internet


Las variables ordinales son:
* Horas de estudio


Debido a que hay múltiples variables tipo objeto y eso no puede ser usado para hacer un modelo de regresión, se decide hacer transformaciones a estas variables.
Para comenzar, se cambia la columna de **"Sexo"** a 1 y 0 para poder trabajar con ellos. Se crea una nueva columna 'sexo_M' donde el 0 representa "Mujer" y el 1 representa "Hombre" con la función `get_dummies()`. Esto ayudará ya que en el modelo se podrá definir por cuántas unidades cambia la calificación si el estudiante es hombre con respecto a si es mujer. Enseguida, por la misma razón y con la misma metodología, se cambia a 1 y 0 la columna de **"Escuela"**. De igual manera, se hace lo mismo para la categoría **"Internet"**
Para que el modelo no asuma una falsa linealidad en la columna de HorasDeEstudio también se decide transformar estos valores. Se trató como variable categórica y se transformó mediante codificación dummy. Para hacerlo de manera más eficiente se utiliza un `for` loop y sólo se hace con los objetos que sean de tipo "objeto". Para asegurar que no se vaya a crear un error por querer hacer un modelo de regresión lineal con variables tipo "object" se utiliza la función `astype(float)` para convertirlos todos a números reales (en vez de *True* y *False*).  


Posteriormente se separan los datos en datos de prueba (20%) y datos de entrenamiento (80%), importando la librería `train_test_split`. Para después continuar con la realización de un modelo, se importa la librería `statsmodels.api`. Para asegurar que siempre se partan los datos de la misma manera cada vez que se ejecute el código se utiliza la semilla `random_state=42`.  Después se guardaron los datos de la calificación final **G3** en la variable `Y` y todos los demás en la variable `X`, usando `.drop()` para no incluir la variable **G3**.


Después de esto se realiza el modelo estimando los coeficientes usando Mínimos Cuadrados Ordinarios con la función `model.fit()`. Luego se muestran los resultados estadísticos para poder analizarlos con la función `results.summary()`.







In [202]:
import numpy as np
ind=np.where(datos.dtypes==object)
for i in ind:
      dummy=pd.get_dummies(datos.iloc[:,i],drop_first=True)
      datos=pd.concat([datos,dummy],axis=1)
datos=datos.drop(labels=datos.columns[ind],axis=1)
print(datos.head())
datos = datos.astype(float)

from sklearn.model_selection import train_test_split
train, test = train_test_split(datos, train_size = 0.8, random_state=42)

import statsmodels.api as sm
X = train.drop('G3', axis = 1)
Y = train['G3']


model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()
print(results.summary())

   Edad  HorasDeEstudio  Reprobadas  Faltas  G1  G2  G3  Escuela_MS  Sexo_M  \
0    18               2           0       6   5   6   6       False   False   
1    17               2           0       4   5   5   6       False   False   
2    15               2           3      10   7   8  10       False   False   
3    15               3           0       2  15  14  15       False   False   
4    16               2           0       4   6  10  10       False   False   

   Internet_yes  
0         False  
1          True  
2          True  
3          True  
4         False  
                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     180.4
Date:                Tue, 03 Feb 2026   Prob (F-statistic):          9.90e-117
Time:                 

Se puede observar que el valor de la R cuadrada ajustada es bastante bueno, obteniendo un valor de 0.837 y múltiples variables tienen un p-value mayor a 0.05. Sólo la **Edad**, cantidad de **Reprobadas**, las **Faltas**, la calificación **G1** y la calificación **G2** tienen un p-value menor a 0.05, sugiriendo que estas son variables relevantes para predecir la calificación final **G3**.


Para identificar posibles problemas de colinealidad o redundancia de información se usa la función `corr()` para calcular la correlación entre los valores numéricos en la base de datos y así poder identificar posibles variables a eliminar. Se agrega la función `abs` ya que sólo se busca conocer la magnitud de esta correlación, no la dirección.







In [203]:
corr = X.corr().abs()
print(corr)

                    Edad  HorasDeEstudio  Reprobadas    Faltas        G1  \
Edad            1.000000        0.032411    0.235018  0.154107  0.028267   
HorasDeEstudio  0.032411        1.000000    0.129183  0.029558  0.202975   
Reprobadas      0.235018        0.129183    1.000000  0.023092  0.346077   
Faltas          0.154107        0.029558    0.023092  1.000000  0.004218   
G1              0.028267        0.202975    0.346077  0.004218  1.000000   
G2              0.116579        0.153854    0.335638  0.007070  0.833365   
Escuela_MS      0.352029        0.056436    0.063895  0.101744  0.007185   
Sexo_M          0.017898        0.333974    0.043433  0.114845  0.054749   
Internet_yes    0.095451        0.045546    0.045589  0.109620  0.098053   

                      G2  Escuela_MS    Sexo_M  Internet_yes  
Edad            0.116579    0.352029  0.017898      0.095451  
HorasDeEstudio  0.153854    0.056436  0.333974      0.045546  
Reprobadas      0.335638    0.063895  0.043433    

Después de correr estas líneas de código se percibe que las únicas variables que tienen un valor de correlación mayor a 0.8 son G1 y G2. Sin embargo, al hacer un análisis considerando el contexto se decide no eliminar ninguna de estas variables. Aunque ambas son calificaciones, cada una mide el desempeño de distintos periodos del semestre por lo que ambos valores aportan información sobre el desempeño académico relevante para predecir la calificación final. Además, como se mencionó anteriormente, ambos tienen p-values menores a 0.05.


Sin embargo, hay variables como **Internet_yes**, **HorasDeEstudio**, **Sexo_M** y **Escuela_MS** que al tener un p-value tan alto no sea conveniente mantener, ya que su contribución al modelo sería bastante limitada.


Debido al p-value menor a 0.05, se conservan las variables **Edad**, **Reprobadas**, **Faltas**, **G1** y **G2**.


Para crear un nuevo modelo con sólo estas variables se volverá a crear una variable `X` con sólo los datos seleccionados. Después se vuelve a separar los datos en `X_train`, `X_test`, `Y_train`, `Y_test` para más tarde poder comprobar la efectividad del modelo. Esto se logra con la función `train_test_split()`, guardando el 80% de los datos como datos de entrenamiento y agregando la misma semilla que en la separación anterior.
De la misma manera que se realizó el modelo anterior se hace este, imprimiendo al final el `summary` para poder analizarlo mejor.





In [204]:
selec = ['Edad', 'Reprobadas','Faltas', 'G1', 'G2']
X = datos[selec]
Y = datos['G3']

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, train_size=0.8, random_state=42
)

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model_selec = sm.OLS(Y_train, X_train_const)
results = model_selec.fit()


print(results.summary())




                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     325.6
Date:                Tue, 03 Feb 2026   Prob (F-statistic):          4.72e-121
Time:                        00:15:06   Log-Likelihood:                -639.86
No. Observations:                 316   AIC:                             1292.
Df Residuals:                     310   BIC:                             1314.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1493      1.467      0.783      0.4

Este nuevo modelo con 5 variables tiene un valor de R cuadrada ajustada de 0.837, igual que el obtenido en el modelo que contenía todas las variables. Esto quiere decir que las variables **Internet_yes**, **HorasDeEstudio**, **Sexo_M** y **Escuela_MS** eran irrelevantes o redundantes y que con las variables que se mantuvieron eran suficientes para explicar el comportamiento de **G3**.


De nuevo, estas 5 variables tienen un p-value menor a 0.05, lo cual indica que sí son significativos para este modelo. La variable **G2** es la que mayor peso tiene en el modelo, ya que con un un punto de calificación que suba esta variable, la calificación final subirá 0.96 puntos. Las variables que muestran una relación negativa con la calificación final son la **Edad** y las materias **Reprobadas**. El hecho de que conforme más edad tienen peor les va en calificaciones puede deberse a que algunos tienen que comenzar a trabajar y tienen que dividir su tiempo entre más labores. Además, también podrían afectar temas como pubertad/adolescencia que puede hacer que pierdan el interés por ponerle esfuerzo a la escuela. Y para las materias reprobadas, puede esperarse que alguien que reprobó una materia no quiera/pueda mostrar interés por la escuela y obtener buenas calificaciones.


Ahora, para ver qué tan bueno es el modelo con datos reales, se hacen predicciones utilizando los datos de prueba y los datos de entrenamiento. Esto se logra con la función `.predict()`.


Después se importa la librería `mean_squared_error, r2_score` para poder obtener la media cuadrada (que sirve para medir qué tan lejos están las predicciones de los valores reales, en promedio) y la R2 (que mide qué porcentaje de la variabilidad de G3 es explicado por el modelo). Se usan las funciones `mean_squared_error()` y `r2_score`.
Posteriormente se obtiene el valor promedio de los cuadrados de las diferencias entre los valores reales y los predichos y el valor de la proporción de la varianza explicada en los datos de entrenamiento.  


Finalmente, se pone a prueba el modelo obteniendo los mismos valores (MSE y R2) pero usando los datos de prueba, a los cuales el modelo nunca tuvo acceso. Después de eso se imprimen los valores obtenidos para poder hacer el análisis y determinar qué tan bueno es el modelo.







In [205]:
# Predicciones
Y_train_pred = results.predict(X_train_const)
Y_test_pred = results.predict(X_test_const)

from sklearn.metrics import mean_squared_error, r2_score

# Entrenamiento
mse_train = mean_squared_error(Y_train, Y_train_pred)
r2_train = r2_score(Y_train, Y_train_pred)

# Prueba
mse_test = mean_squared_error(Y_test, Y_test_pred)
r2_test = r2_score(Y_test, Y_test_pred)

print(f"Entrenamiento: MSE={mse_train:.2f}, R²={r2_train:.2f}")
print(f"Prueba: MSE={mse_test:.2f}, R²={r2_test:.2f}")

Entrenamiento: MSE=3.36, R²=0.84
Prueba: MSE=4.51, R²=0.78


Después de realizar estos cálculos se puede ver que el modelo explica el 78% de los datos que no vio durante el entrenamiento. Además, en promedio los datos se desvían 2.1 puntos. Esto indica que, en general, el modelo creado puede predecir con suficiente certeza el valor de la calificación final **G3** utilizando las variables **Edad**, **Reprobadas**, **Faltas**, **G1** y **G2**.


Finalmente, tomando en cuenta los valores de R cuadrada obtenidos para los datos de entrenamiento y de prueba, el modelo creado explicar entre el 78-84% de la variación de la calificación final de los alumnos.






---


Al hacer este reporte resaltó la importancia de hacer una adecuada preparación de datos antes de entrenar al modelo de regresión lineal múltiple. La transformación de variables categóricas a variables dummy y posteriormente transformar todos los valores a numéricos fue fundamental para poder crear el modelo y que no hubiera errores en el proceso. Asimismo, la separación de la base de datos en subconjuntos de entrenamiento y prueba fue un paso muy importante para poder evaluar realmente el desempeño del modelo y evitar la fuga de datos.


El análisis de correlación entre variables y el p-value facilitó la selección de características relevantes, ya que de esta manera se logró reducir la cantidad de variables significativas incluidas en la creación del modelo para la predicción de la calificación final. Se observó que el desempeño académico previo y compromiso con la asistencia (G1, G2, Reprobadas y Faltas), al igual que la Edad, explica una proporción considerable de la variabilidad de las calificaciones finales. La estabilidad de la R cuadrada ajustada después de la eliminación de las variables no relevantes sugiere que dichas variables no aportaban información relevante al modelo.


El modelo final mostró un buen desempeño tanto en el conjunto de datos de entrenamiento como en los de prueba, con valores alto de R cuadrada  errores moderados, lo cuál implica una adecuada capacidad de generalización y ausencia de sobreajuste significativo. A pesar de esto, el modelo tiene ciertas limitaciones. Una de estas siendo que el modelo asume relaciones lineales entre las variables de entrada y la variable de salida. Así mismo, no se consideran posibles interacciones entre las variables que puedan mejorar el modelo.


Como posibles mejoras futuras se podría explorar la incorporación de términos de interacción, modelos no lineales y el uso de distintas técnicas de regularización. Finalmente, añadir variables adicionales como factores socioeconómicos u horas de sueño podría contribuir a una mejor comprensión y predicción del comportamiento académico de un estudiante.


**Código de Honor**: Doy mi palabra que he realizado esta actividad con integridad académica.  


Se implementó el uso de IA únicamente para dudas de redacción y código.





